In [ ]:
# Monter Google Drive pour sauvegarder les modèles
from google.colab import drive
drive.mount('/content/drive')

# Installer PyTorch et autres bibliothèques nécessaires
#!pip install torch torchvision numpy matplotlib
#!pip install trimesh
#!pip install tqdm


# Télécharger et extraire le dataset Pix3D

!wget http://pix3d.csail.mit.edu/data/pix3d.zip
!unzip pix3d.zip && rm pix3d.zip


# **Importation des bibliothèques nécessaires:**

---


### Nous commençons par importer toutes les bibliothèques nécessaires pour le traitement des données, la gestion des modèles, et la manipulation des fichiers.

In [ ]:
# Importation des bibliothèques nécessaires
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import scipy.io as sio  # Pour charger les fichiers .mat
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image  # Charger l'image using PIL instead of matplotlib
from torch.nn.utils.rnn import pad_sequence


# Configuration de l'appareil (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de l'appareil : {device}")


Utilisation de l'appareil : cpu


# **Étape 2 : Définition du Dataset Pix3D**

---


### Classe Pix3DDataset :
#### Cette classe permet de charger les images 2D, masques, voxels, keypoints, et les modèles 3D basés sur le fichier JSON.

In [ ]:
class Pix3DDataset(Dataset):
    def __init__(self, dataset_path, json_file, transform=None):
        self.dataset_path = Path(dataset_path)
        self.json_file = self.dataset_path / json_file
        self.transform = transform
        self.data = self.load_metadata()

    def load_metadata(self):
        """Charge les métadonnées depuis le fichier JSON."""
        with open(self.json_file, 'r') as f:
            metadata = json.load(f)
        data = []
        for entry in metadata:
            img_path = self.dataset_path / entry['img']
            mask_path = self.dataset_path / entry['mask']
            model_path = self.dataset_path / entry['model']
            voxel_path = self.dataset_path / entry['voxel']
            keypoint_path = self.dataset_path / entry['3d_keypoints']
            if img_path.exists() and model_path.exists():
                data.append({
                    'img': img_path,
                    'mask': mask_path,
                    'model': model_path,
                    'voxel': voxel_path,
                    'keypoints': keypoint_path
                })
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        img = Image.open(entry['img']).convert('RGB')  # Assurer que l'image est en format RGB
        if self.transform:
            img = self.transform(img)

        # Charger le masque (si disponible)
        mask = Image.open(entry['mask']).convert('L') if entry['mask'].exists() else None
        if self.transform and mask is not None:
            mask = self.transform(mask)

        # Charger les voxels (format .mat)
        voxel = sio.loadmat(entry['voxel'])['voxel'] if entry['voxel'].exists() else None

        # Charger les keypoints 3D
        keypoints = np.loadtxt(entry['keypoints']) if entry['keypoints'].exists() else None

        return {
            'img': img,
            'mask': mask,
            'voxel': voxel,
            'keypoints': keypoints,
            'model': entry['model']
        }


### La fonction `collate_fn` est utilisée pour préparer les données lors de la création de batches dans un DataLoader personnalisé en PyTorch.Cette fonction est essentielle pour manipuler des données de tailles variables dans des modèles de deep learning.


---

# **Objectif**
#### Elle combine les données d'un batch en les rendant compatibles, en particulier en ajoutant un padding uniforme aux keypoints (points clés) pour garantir que toutes les entrées ont la même taille.

In [ ]:
def collate_fn(batch):
    """Fonction pour combiner les éléments d'un batch et ajouter un padding aux keypoints."""
    max_keypoints = max([len(d['keypoints']) for d in batch if d['keypoints'] is not None])

    keypoints = []
    for d in batch:
        if d['keypoints'] is not None:
            padded_keypoints = torch.cat([torch.tensor(d['keypoints']), torch.zeros(max_keypoints - len(d['keypoints']), 3)], dim=0)
            keypoints.append(padded_keypoints)
        else:
            keypoints.append(torch.zeros(max_keypoints, 3))  # Padding complet si aucun keypoint

    keypoints = torch.stack(keypoints)

    return {
        'img': torch.stack([d['img'] for d in batch]),
        'mask': torch.stack([d['mask'] for d in batch]) if batch[0]['mask'] is not None else None,
        'voxel': torch.stack([torch.tensor(d['voxel'], dtype=torch.float32) for d in batch]),
        'keypoints': keypoints,
        'model': [d['model'] for d in batch]
    }


# **Étape 3 : Définition des modèles GAN**

---

## **Générateur :**

### Le générateur prend une image 2D en entrée et produit un volume 3D

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.ReLU()
        )
        self.fc = nn.Linear(256 * 8 * 8, 32 * 32 * 32)
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(1, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 1, 4, stride=2, padding=1),  # Changez 1 en 16 si vous voulez plusieurs canaux
            nn.Tanh()  # Ou une autre fonction d'activation appropriée
        )

    def forward(self, x):
        features = self.encoder(x)
        flat = features.view(features.size(0), -1)
        volume = self.fc(flat).view(-1, 1, 32, 32, 32)  # Changez 1 en 16 si vous voulez plusieurs canaux
        output = self.decoder(volume)
        return output


# **Discriminateur**

---


### Le discriminateur évalue si un volume 3D est réel ou généré.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=4, stride=2, padding=1), # Changed input channels to 1
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(512, 1, kernel_size=4, stride=2, padding=1),
        )
        self.fc = nn.Linear(4*4*4, 1)  # Adjusted linear layer input size to match flattened output

    def forward(self, x):
        out = self.net(x)
        #print(f"Forme avant flatten: {out.shape}")
        out = out.view(out.size(0), -1)  # Flatten
        #print(f"Forme après flatten: {out.shape}")
        out = self.fc(out)
        out = torch.sigmoid(out)
        return out

# **Étape 4 :**
---
### Fonction pour sauvegarder des modèles 3D au format .obj



In [ ]:
def save_as_obj(voxel, file_path):
    """Sauvegarde un volume voxelisé en format .obj."""
    voxel = voxel.squeeze().cpu().numpy()  # Convertir en NumPy
    with open(file_path, 'w') as f:
        for x in range(voxel.shape[0]):
            for y in range(voxel.shape[1]):
                for z in range(voxel.shape[2]):
                    if voxel[x, y, z] > 0.5:  # Seuil pour considérer comme occupé
                        f.write(f"v {x} {y} {z}\n")


# **Étape 5 :**
---
## Boucle d’entraînement


In [ ]:
# Initialisation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = Generator().to(device)
D = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_G = optim.Adam(G.parameters(), lr=0.0002)
optimizer_D = optim.Adam(D.parameters(), lr=0.0002)

# Charger le dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = Pix3DDataset('/content/pix3d', 'pix3d.json', transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# Chemins pour sauvegarder les modèles et les logs
output_dir = './output_models'
os.makedirs(output_dir, exist_ok=True)
log_file = os.path.join(output_dir, 'training_logs.txt')

# Initialisation du fichier de log
with open(log_file, 'w') as f:
    f.write("Epoch\tD_Loss\tG_Loss\n")

# Réduction des besoins en ressources et des logs
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Époque {epoch + 1}/{num_epochs}")
    epoch_d_loss, epoch_g_loss = 0.0, 0.0
    for i, batch in enumerate(dataloader):
        # Chargement des données
        real_images = batch['img'].to(device)
        real_voxels = batch['voxel'].to(device).float().unsqueeze(1)

        # Vérifiez les dimensions des données
        assert real_images.dim() == 4, f"Problème avec real_images : {real_images.shape}"
        assert real_voxels.dim() == 5, f"Problème avec real_voxels : {real_voxels.shape}"

        # Mise à jour du discriminateur
        optimizer_D.zero_grad()
        real_labels = torch.ones(real_images.size(0), 1).to(device)
        fake_labels = torch.zeros(real_images.size(0), 1).to(device)

        # Perte réelle
        outputs_real = D(real_voxels)
        d_loss_real = criterion(outputs_real, real_labels)

        # Perte fausse
        with torch.no_grad():
            fake_voxels = G(real_images)
        outputs_fake = D(fake_voxels.detach())
        d_loss_fake = criterion(outputs_fake, fake_labels)

        # Mise à jour
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        # Mise à jour du générateur
        optimizer_G.zero_grad()
        outputs_fake = D(fake_voxels)
        g_loss = criterion(outputs_fake, real_labels)

        # Diagnostic avant le calcul des gradients
        assert not torch.isnan(g_loss), "g_loss contient NaN"
        g_loss.backward()
        optimizer_G.step()

        # Suivi des pertes
        epoch_d_loss += d_loss.item()
        epoch_g_loss += g_loss.item()

        if (i + 1) % 50 == 0:  # Log moins souvent
            print(f"Époque [{epoch+1}/{num_epochs}], Étape [{i+1}/{len(dataloader)}], "
                  f"D_Loss: {d_loss.item():.4f}, G_Loss: {g_loss.item():.4f}")

    # Pertes moyennes
    avg_d_loss = epoch_d_loss / len(dataloader)
    avg_g_loss = epoch_g_loss / len(dataloader)
    print(f"\n=> Fin de l'époque [{epoch+1}/{num_epochs}] - D_Loss: {avg_d_loss:.4f}, G_Loss: {avg_g_loss:.4f}\n")

    # Sauvegarde des logs
    with open(log_file, 'a') as f:
        f.write(f"{epoch+1}\t{avg_d_loss:.4f}\t{avg_g_loss:.4f}\n")

    # Sauvegarde d'un modèle 3D généré
    sample_voxel = fake_voxels[0]
    obj_path = os.path.join(output_dir, f'generated_model_epoch_{epoch+1}.obj')
    save_as_obj(sample_voxel, obj_path)

    print(f"Modèle 3D sauvegardé : {obj_path}")
    print(f"Logs mis à jour dans {log_file}")

# Sauvegarde de l'état final du modèle générateur
generator_model_path = os.path.join(output_dir, 'generator_final.pth')
torch.save(G.state_dict(), generator_model_path)
print(f"L'état du modèle générateur a été sauvegardé à : {generator_model_path}")

Époque 1/50


# **Utilisation du générateur sauvegardé pour la génération**

---



### Pour utiliser le modèle générateur sauvegardé, chargez son état avec `torch.load` et générez un nouveau modèle 3D.

### Code pour charger le générateur et générer un modèle

In [ ]:
# Charger le modèle générateur sauvegardé
G_loaded = Generator().to(device)
G_loaded.load_state_dict(torch.load(generator_model_path))
G_loaded.eval()  # Mettre le modèle en mode évaluation

# Exemple de génération avec une nouvelle image
def generate_3d_model(image_path, generator, output_path):
    """
    Génère un modèle 3D à partir d'une image 2D et le sauvegarde en .obj.
    :param image_path: Chemin de l'image d'entrée.
    :param generator: Modèle générateur chargé.
    :param output_path: Chemin pour sauvegarder le fichier .obj généré.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 128)),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = plt.imread(image_path)
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        generated_voxel = generator(image)
        save_as_obj(generated_voxel, output_path)

# Générer un modèle 3D avec une nouvelle image
new_image_path = '/content/pix3d/img/bed/0001.png'
output_3d_path = './generated_new_model.obj'
generate_3d_model(new_image_path, G_loaded, output_3d_path)
print(f"Modèle 3D généré et sauvegardé à : {output_3d_path}")
